In [15]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
#This is a custom function that writes all the decision variables to an excel file.
#from Helper_Functions.exportDecisionVariables import exportDecisionVariables


def readInputFile(filename):
    NetworkData = pd.read_excel(filename, sheet_name= 'NetworkData', index_col=0)
    SystemDemand= pd.read_excel(filename, sheet_name = 'SystemDemand', index_col= 0)
    PVGeneration = pd.read_excel(filename, sheet_name='PVGeneration', index_col=0)
    WindGeneration = pd.read_excel(filename, sheet_name='WindGeneration', index_col=0) 
    LoadData = pd.read_excel(filename, sheet_name = 'Loads', index_col= 0)
    PVData = pd.read_excel(filename, sheet_name='PVParks', index_col=0)
    WPData = pd.read_excel(filename, sheet_name='WindParks', index_col=0)
    StorageData = pd.read_excel(filename, sheet_name= 'StorageSystems', index_col=0)
    UnitData = pd.read_excel(filename, sheet_name = 'Generators', index_col= 0)
    GeneratorStepSizeData = pd.read_excel(filename, sheet_name = 'GeneratorStepSize', index_col= 0)
    GeneratorStepCostData = pd.read_excel(filename, sheet_name = 'GeneratorStepCost', index_col= 0)

    # We could return multiple items but it is preferable to return a structure, in this case a dictionary 
    return {'NetworkData':NetworkData, 'SystemDemand':SystemDemand, 'PVGeneration':PVGeneration, 'WindGeneration':WindGeneration,
            'LoadData':LoadData, 'PVData':PVData, 'WPData':WPData,'StorageData':StorageData,'UnitData':UnitData,
            'GeneratorStepSizeData':GeneratorStepSizeData, 'GeneratorStepCostData':GeneratorStepCostData}

In [16]:
validation_data = readInputFile(r'Input_Files\Assignment3_ValidationSystem.xlsx')
data = readInputFile(r'Input_Files\Assignment3_InputData.xlsx')

for i in data.keys():
    print(i, ':\n', validation_data[i])
    print('\n')

NetworkData :
        FROM    TO       X  Capacity
Line1  Bus1  Bus2  0.0146      1000
Line2  Bus1  Bus4  0.2253      1000
Line3  Bus2  Bus4  0.0907        30
Line4  Bus2  Bus3  0.1356       100
Line5  Bus3  Bus4  0.2050      1000


SystemDemand :
     SystemDemand
t1           180
t2           185
t3           190
t4           200
t5           215
t6            20


PVGeneration :
     PVGeneration
t1             0
t2             5
t3            10
t4            15
t5             5
t6             0


WindGeneration :
     WindGeneration
t1              10
t2               5
t3               0
t4               5
t5              10
t6              10


LoadData :
       Location  Percentage
Load1     Bus2         0.6
Load2     Bus4         0.4


PVData :
     Location  Percentage
PV1     Bus1           1


WPData :
     Location  Percentage
WP1     Bus2           1


StorageData :
       Power  Energy  SOEini  Eff Location
ESS1      5      10      10  0.9     Bus1


UnitData :
    Type 

In [17]:
def optimizationModel(inputData, modelType):

    NetworkData = inputData['NetworkData']
    SystemDemand= inputData['SystemDemand']
    PVGeneration = inputData['PVGeneration']
    WindGeneration = inputData['WindGeneration']
    LoadData = inputData['LoadData']
    PVData = inputData['PVData']
    WPData = inputData['WPData']
    StorageData = inputData['StorageData']
    UnitData = inputData['UnitData']
    GeneratorStepSizeData = inputData['GeneratorStepSizeData']
    GeneratorStepCostData = inputData['GeneratorStepCostData']

    #---------------------------------------------------------------------------------------------------------
    #Define the Model
    #---------------------------------------------------------------------------------------------------------

    model = ConcreteModel()

    #---------------------------------------------------------------------------------------------------------
    #Define Sets
    #---------------------------------------------------------------------------------------------------------

    model.I = Set(ordered=True, initialize=UnitData.index)
    model.T = Set(ordered=True, initialize=SystemDemand.index)
    model.F = Set(ordered=True, initialize=GeneratorStepSizeData.columns)
    model.S = Set(ordered=True, initialize=StorageData.index)
    model.J = Set(ordered=True, initialize=LoadData.index)
    model.L = Set(ordered=True, initialize=NetworkData.index)
    buses = NetworkData['FROM'].append(NetworkData['TO']).unique()
    model.N = Set(ordered=True, initialize=buses)
    model.K = Set(ordered=True, initialize=PVData.index)
    model.Q = Set(ordered=True, initialize=WPData.index)

    #---------------------------------------------------------------------------------------------------------
    #Define Parameters
    #---------------------------------------------------------------------------------------------------------

    # Conventional generating units
    model.Pmin = Param(model.I, within=NonNegativeReals, mutable=True)
    model.Pmax = Param(model.I, within=NonNegativeReals, mutable=True)
    model.RU = Param(model.I, within=NonNegativeReals, mutable=True)
    model.RD = Param(model.I, within=NonNegativeReals, mutable=True)
    model.SUC = Param(model.I, within=NonNegativeReals, mutable=True)
    model.SDC = Param(model.I, within=NonNegativeReals, mutable=True)
    model.Pini = Param(model.I, within=NonNegativeReals, mutable=True)
    model.uini = Param(model.I, within=Binary, mutable=True)
    model.C = Param(model.I, model.F, within=NonNegativeReals, mutable=True)
    model.B = Param(model.I, model.F, within=NonNegativeReals, mutable=True)

    # Load
    model.D = Param(model.J, model.T, within=NonNegativeReals, mutable=True)

    # PV parks
    model.PV = Param(model.K, model.T, within=NonNegativeReals, mutable=True)

    # Wind parks
    model.W = Param(model.Q, model.T, within=NonNegativeReals, mutable=True)

    # Energy storage
    model.PEESmax = Param(model.S, within=NonNegativeReals, mutable=True)
    model.SOEmax = Param(model.S, within=NonNegativeReals, mutable=True)
    model.SOEini = Param(model.S, within=NonNegativeReals, mutable=True)
    model.eff = Param(model.S, within=NonNegativeReals, mutable=True)

    # Network
    model.X = Param(model.L, within=NonNegativeReals, mutable=True)
    model.Pline_max = Param(model.L, within=NonNegativeReals, mutable=True)
    model.Bmat = Param(model.L, model.N, within=Reals, mutable=True)

    #---------------------------------------------------------------------------------------------------------
    #Initialize Parameters
    #---------------------------------------------------------------------------------------------------------

    # Conventional generating units
    for i in model.I:
        model.Pmin[i] = UnitData.loc[i, 'Pmin']
        model.Pmax[i] = UnitData.loc[i, 'Pmax']
        model.RU[i] = UnitData.loc[i, 'RU']
        model.RD[i] = UnitData.loc[i, 'RD']
        model.SUC[i] = UnitData.loc[i, 'SUC']
        model.SDC[i] = UnitData.loc[i, 'SDC']
        model.Pini[i] = UnitData.loc[i, 'Pini']
        model.uini[i] = UnitData.loc[i, 'uini']
        for f in model.F:
            model.C[i, f] = GeneratorStepCostData.loc[i, f]
            model.B[i, f] = GeneratorStepSizeData.loc[i, f]

    # Load
    for j in model.J:
        for t in model.T:
            model.D[j, t] = SystemDemand.loc[t, 'SystemDemand']*LoadData.loc[j, 'Percentage']

    # PV parks
    for k in model.K:
        for t in model.T:
            model.PV[k, t] = PVGeneration.loc[t, 'PVGeneration']*PVData.loc[k, 'Percentage']

    # Wind parks
    for q in model.Q:
        for t in model.T:
            model.W[q, t] = WindGeneration.loc[t, 'WindGeneration']*WPData.loc[q, 'Percentage']

    # Energy storage
    for s in model.S:
        model.PEESmax[s] = StorageData.loc[s, 'Power']
        model.SOEmax[s] = StorageData.loc[s, 'Energy']
        model.SOEini[s] = StorageData.loc[s, 'SOEini']
        model.eff[s] = StorageData.loc[s, 'Eff']

    # Network
    for l in model.L:
        model.X[l] = NetworkData.loc[l, 'X']
        model.Pline_max[l] = NetworkData.loc[l, 'Capacity']
        for n in model.N:
            if NetworkData.loc[l, 'FROM'] == n:
                model.Bmat[l, n] = 1/model.X[l]
            elif NetworkData.loc[l, 'TO'] == n:
                model.Bmat[l, n] = -1/model.X[l]
            else:
                model.Bmat[l, n] = 0

    #---------------------------------------------------------------------------------------------------------
    #Define Decision Variables
    #---------------------------------------------------------------------------------------------------------

    # Conventional generating units
    model.P = Var(model.I, model.T, within=NonNegativeReals)
    model.b = Var(model.I, model.F, model.T, within=NonNegativeReals)
    model.u = Var(model.I, model.T, within=Binary)
    model.CSU = Var(model.I, model.T, within=NonNegativeReals)
    model.CSD = Var(model.I, model.T, within=NonNegativeReals)

    # Energy storage
    model.SOE = Var(model.S, model.T, within=NonNegativeReals)
    model.Pch = Var(model.S, model.T, within=NonNegativeReals)
    model.Pdis = Var(model.S, model.T, within=NonNegativeReals)
    model.u_ees = Var(model.S, model.T, within=Binary)

    # Network
    model.f = Var(model.L, model.T, within=Reals)
    model.theta = Var(model.N, model.T, within=Reals)

    #---------------------------------------------------------------------------------------------------------
    #Define Constraints
    #---------------------------------------------------------------------------------------------------------

    # Objective function
    def objective_cost_rule(model):
        return sum(sum(sum(model.C[i,f]*model.b[i,f,t] for f in model.F) + \
               model.CSU[i,t] + model.CSD[i,t] for i in model.I) for t in model.T)

    # Generating units
    def power_decomposition_rule1(model, i, t):
        return sum(model.b[i,f,t] for f in model.F) == model.P[i,t]

    def power_decomposition_rule2(model, i, f, t):
        return model.b[i,f,t] <= model.B[i,f]

    # def power_minmax_rule(model, i, t):
    #     return inequality(model.Pmin[i]*model.u[i,t], model.P[i,t], model.Pmax[i]*model.u[i,t])

    def PowerMin(model, i , t): #Eq. (4)
        return model.P[i,t] >= model.Pmin[i]* model.u[i,t]

    def PowerMax(model, i, t): #Eq. (4)
        return model.P[i,t] <= model.Pmax[i]* model.u[i,t]

    def rampUp_rule(mode, i, t):
        if model.T.ord(t) == 1:
            return model.P[i, t] - model.Pini[i] <= 60*model.RU[i]
        if model.T.ord(t) > 1:
            return model.P[i, t] - model.P[i, model.T.prev(t)] <= 60*model.RU[i]

    def rampDown_rule(model, i, t):
        if model.T.ord(t) == 1:
            return model.Pini[i] - model.P[i, t] <= 60*model.RD[i]
        if model.T.ord(t) > 1:
            return model.P[i, model.T.prev(t)] - model.P[i, t] <= 60*model.RD[i]

    def startUpCost_rule(model, i, t):
        if model.T.ord(t) == 1:
            return model.CSU[i,t] >= model.SUC[i]*(model.u[i,t] - model.uini[i])
        if model.T.ord(t) > 1:
            return model.CSU[i,t] >= model.SUC[i]*(model.u[i,t] - model.u[i, model.T.prev(t)])

    def startDownCost_rule(model, i, t):
        if model.T.ord(t) == 1:
            return model.CSD[i,t] >= model.SDC[i]*(model.uini[i] - model.u[i,t])
        if model.T.ord(t) > 1:
            return model.CSD[i,t] >= model.SDC[i]*(model.u[i, model.T.prev(t)] - model.u[i,t])

    # EES
    def state_of_energy_rule(model, s, t):
        if model.T.ord(t) == 1:
            return model.SOE[s,t] == model.SOEini[s] + model.eff[s]*model.Pch[s,t] - \
                   model.Pdis[s,t]/model.eff[s]
        if model.T.ord(t) > 1:
            return model.SOE[s,t] == model.SOE[s, model.T.prev(t)] + model.eff[s]*model.Pch[s,t] - \
                   model.Pdis[s,t]/model.eff[s]

    def soc_minmax_rule(model, s, t):
        return model.SOE[s,t] <= model.SOEmax[s]

    def charge_ees_rule(model, s, t):
        return model.Pch[s,t] <= model.PEESmax[s]*model.u_ees[s,t]

    def discharge_ees_rule(model, s, t):
        return model.Pdis[s,t] <= model.PEESmax[s]*(1 - model.u_ees[s,t])

    # Network
    def theta_ini_rule(model, t):
        return model.theta['Bus1', t] == 0

    def flow_minmax_rule(model, l, t):
        return inequality(-model.Pline_max[l], model.f[l,t], model.Pline_max[l])

    # define flow through line l betwen bus n1 and n2 at time t (only when are connected)
    def line_flow_rule(model, l, t, n1, n2):
        if ((n1 == NetworkData.loc[l,'FROM']) and (n2 == NetworkData.loc[l,'TO'])) \
            or ((n1 == NetworkData.loc[l,'TO']) and (n2 == NetworkData.loc[l,'FROM'])):
            return model.f[l,t] == model.Bmat[l,n1] * (model.theta[n1,t]-model.theta[n2,t])
        else:
            return Constraint.Skip

    # Nodal Power Balance
    def nodal_balance_rule(model, n, t):
        return sum(model.PV[k,t] for k in model.K if PVData.loc[k, 'Location'] == n) + \
               sum(model.W[q,t] for q in model.Q if WPData.loc[q, 'Location'] == n) + \
               sum(model.Pdis[s,t] for s in model.S if StorageData.loc[s, 'Location'] == n) + \
               sum(model.P[i,t] for i in model.I if UnitData.loc[i, 'Location'] == n) + \
               sum(model.f[l,t] for l in model.L if NetworkData.loc[l, 'TO'] == n) == \
               sum(model.D[j,t] for j in model.J if LoadData.loc[j, 'Location'] == n) + \
               sum(model.Pch[s,t] for s in model.S if StorageData.loc[s, 'Location'] == n) + \
               sum(model.f[l,t] for l in model.L if NetworkData.loc[l, 'FROM'] == n)

    def balance(model, t):
        return sum(model.PV[k,t] for k in model.K)+\
               sum(model.W[q,t] for q in model.Q)+\
               sum(model.P[i,t] for i in model.I) + sum(model.Pdis[s,t] for s in model.S) \
               == sum(model.D[j,t] for j in model.J) + sum(model.Pch[s,t] for s in model.S)

    # Assign objective and constraints to the model
    model.objective_cost = Objective(rule=objective_cost_rule)
    model.power_decomposition1 = Constraint(model.I, model.T, rule=power_decomposition_rule1)
    model.power_decomposition2 = Constraint(model.I, model.F, model.T, rule=power_decomposition_rule2)
    #model.power_minmax = Constraint(model.I, model.T, rule=power_minmax_rule)
    model.power_min = Constraint(model.I, model.T, rule=PowerMin)
    model.power_max = Constraint(model.I, model.T, rule=PowerMax)
    model.startUpCost = Constraint(model.I, model.T, rule=startUpCost_rule)
    model.startDownCost = Constraint(model.I, model.T, rule=startDownCost_rule)
    #
    model.state_of_energy = Constraint(model.S, model.T, rule=state_of_energy_rule)
    model.soc_minmax = Constraint(model.S, model.T, rule=soc_minmax_rule)
    model.charge_ees = Constraint(model.S, model.T, rule=charge_ees_rule)
    model.discharge_ees = Constraint(model.S, model.T, rule=discharge_ees_rule)

    if modelType == 'Case1':
        model.bal = Constraint(model.T, rule=balance)

    elif modelType == 'Case2':
        # Ramping Constraints
        model.rampUp = Constraint(model.I, model.T, rule=rampUp_rule)
        model.rampDown = Constraint(model.I, model.T, rule=rampDown_rule)
        # Transmission line limits
        model.flow_minmax = Constraint(model.L, model.T, rule=flow_minmax_rule)
        # Network constraints
        model.theta_ini = Constraint(model.T, rule=theta_ini_rule)
        model.line_flow = Constraint(model.L, model.T, model.N, model.N, rule=line_flow_rule)
        model.nodal_balance = Constraint(model.N, model.T, rule=nodal_balance_rule)

    elif modelType == 'Case3':
        model.rampUp = Constraint(model.I, model.T, rule=rampUp_rule)
        model.rampDown = Constraint(model.I, model.T, rule=rampDown_rule)

        # Network constraints
        model.theta_ini = Constraint(model.T, rule=theta_ini_rule)
        model.line_flow = Constraint(model.L, model.T, model.N, model.N, rule=line_flow_rule)
        model.nodal_balance = Constraint(model.N, model.T, rule=nodal_balance_rule)

    else:
        print('This is not a valid model type!')

    return model

In [18]:
model = optimizationModel(validation_data, 'Case1')

C[G1,f1] + C[G2,f1] + C[G3,f1] + C[G1,f2] + C[G2,f2] + C[G3,f2] + C[G1,f3] + C[G2,f3] + C[G3,f3] + C[G1,f4] + C[G2,f4] + C[G3,f4]
ERROR: Rule failed when generating expression for objective objective_cost:
    KeyError: "Index '('1', 'f')' is not valid for indexed component 'C'"
ERROR: Constructing component 'objective_cost' from data=None failed:
    KeyError: "Index '('1', 'f')' is not valid for indexed component 'C'"


KeyError: "Index '('1', 'f')' is not valid for indexed component 'C'"

In [ ]:
opt=SolverFactory('gurobi')
opt.options["MIPGap"] = 0.0
results=opt.solve(model)
print('Total cost (Euro): ', model.objective_cost())
# exportDecisionVariables(model, 'TestExportCase1.xlsx')

results_df = pd.DataFrame(index=validation_data['UnitData'].index, columns=['Type', 'Total Power [MW]', 'Total Energy [MWd]', 'Total Cost [Euro/day]', 'Daily Commitment Cost [Euro]'], dtype=float)
#B - For each generator type (per type code) compute and comment on:
for i in model.I:
    results_df.loc[i, 'Type'] = validation_data['UnitData'].loc[i,'Type']
    results_df.loc[i, 'Total Power [MW]'] = sum(model.P[i, t].value for t in model.T)
    # the total daily energy generation
    results_df.loc[i, 'Total Energy [MWd]'] = results_df.loc[i, 'Total Power [MW]']
    # the total daily energy costs
    results_df.loc[i, 'Total Cost [Euro/day]'] = sum((sum(model.C[i,f].value*model.b[i,f,t1].value for f in model.F)) for t1 in model.T)
     # the total daily commitment costs (i.e., start-up and shut-down costs)
    results_df.loc[i, 'Daily Commitment Cost [Euro]'] = sum(model.CSU[i, t2].value for t2 in model.T) + sum(model.CSD[i, t3].value for t3 in model.T)
grouped = results_df.groupby('Type')
grouped[['Total Power [MW]','Total Energy [MWd]','Total Cost [Euro/day]','Daily Commitment Cost [Euro]']].sum()


In [ ]:
model = optimizationModel(validation_data, 'Case2')
model.pprint()
opt=SolverFactory('gurobi')
opt.options["MIPGap"] = 0.0
results=opt.solve(model)
print('Total cost (Euro): ', model.objective_cost())
# exportDecisionVariables(model, 'TestExportCase1.xlsx')

In [ ]:
model = optimizationModel(validation_data, 'Case3')
opt=SolverFactory('gurobi')
opt.options["MIPGap"] = 0.0
results=opt.solve(model)
print('Total cost (Euro): ', model.objective_cost())
# exportDecisionVariables(model, 'TestExportCase1.xlsx')